# Admissions markets with a single scoring criterion and MNL choice
Let’s consider a special kind of admissions market that has not received much attention in the school-choice literature but approximates the admissions procedure used in many systems around the world, such as college admissions in South Korea and China. In the *single-score* model, all schools share the same ranking over the students; for example, they care exclusively about students' scores on a specific standardized test. 

In this model, students use the *multinomial logit choice* model. This means that each school has a given *quality* $\delta_c$. Given a set of admitted schools $C^\#$, students attend school $c$ with probability $$\frac{\exp \delta_c}{\sum_{d \in C^\#} \exp \delta_d}$$

For convenience, let $\gamma_c \equiv \exp \delta_c$. Since the equation above is homogeneous in $\gamma$, we may assume without loss of generality that $\sum \gamma_c = 1$.  

Observe that the single-score model is equivalent to the ideal case of single tiebreaking, in which a central agency assigns (by lottery) an admissions priority ranking to each student. Alternatively, the single-score can be interpreted as a linear combination of various dimensions of student characteristics; this is also an accurate reflection of real-world admissions practice, in which schools allocate a certain scoring weight to test scores, grades, and so on.

## Demand function
Without loss of generality, assume schools are ordered by cutoff, i.e. that
$$p_1 \leq p_2 \leq \dots \leq p_{|C|}$$
Then there are only $|C| + 1$ possible consideration sets for each student: 

| Symbol   | Consideration set                 | Probability |
|-----------------|--------------------------------------------|-----------------------|
| $C_{[0]}$       | $\varnothing$                              | $p_1$                 |
| $C_{[1]}$       | $\left\{ c_1 \right\}$                     | $p_2 - p_1$           |
| $C_{[2]}$       | $\left\{ c_1, c_2 \right\}$                | $p_3 - p_2$           |
| $\vdots$        | $\vdots$                                   | $\vdots$              |
| $C_{[|C| - 1]}$ | $\left\{ c_1, \dots, c_{|C| - 1} \right\}$ | $p_{|C|} - p_{|C|-1}$ |
| $C_{[|C|]}$     | $\left\{ c_1, \dots, c_{|C|} \right\}$     | $1 - p_{|C|}$         |

This greatly simplifies the demand function. Letting $p_{|C|+1} \equiv 1$,
$$\begin{equation}D_c(p) = \sum_{d=c}^{|C|} 
\underbrace{\frac{{\gamma_c}}{ \sum_{i=1}^d {\gamma_i}}}_{\substack{\text{prob. of choosing  }\\ c\text{ from assortment}}} 
\overbrace{\left(p_{d+1} - p_{d}\right)}^{\substack{\text{prob. of having}\\ \text{assortment }C_{[d]}}} \end{equation}$$

In [18]:
using LinearAlgebra

"""
Contains static information about a school-choice market.
"""
struct Market
    qualities::Array{<:AbstractFloat, 1}   # δ
    capacities::Array{<:AbstractFloat, 1}  # q
    
    γ = qualities
end

"""
    Market(m)

Generate a random market with m schools.
"""
function Market(m::Int) 
    qualities = rand(m)
    qualities ./= sum(qualities)
    capacities = rand(m)/m
    return Market(qualities, capacities)
end

"""
    demand(market, cutoffs)

Return demand for each school given a set of cutoffs and ignoring capacity, using
multinomial logit choice model with one student profile and a single test score.
"""
function demand(market      ::Market,
                cutoffs     ::AbstractArray{<:AbstractFloat, 1};
                )::AbstractArray{<:AbstractFloat, 1}
    
    m = length(market.qualities)
    demands = zeros(m)

    sort_order = sortperm(cutoffs)
    cutoffs[sort_order]

    γ = exp.(market.qualities)
    demands = zeros(m)

    consideration_set_probabilities = diff([cutoffs[sort_order]; 1])

    for c in 1:m, d in c:m     # For each score threshold
        demands[sort_order[c]] += consideration_set_probabilities[d] *
                                  γ[sort_order[c]] / sum(γ[sort_order[1:d]])
    end

    return demands
end


"""
    quickeq(market, demand)

Very low-tech tatonnement procedure for finding equibilibrium.
"""
function quickeq(market::Market, demand::Function; maxit::Int=100)
    p = rand(length(market.qualities))
    
    for i in 1:maxit
        p = max.(0, p + .5 * (demand(market, p) - market.capacities) / i^0.001)
#         @show p
    end
    
    return p
end

LoadError: syntax: "γ = qualities" inside type definition is reserved around In[18]:3

In [17]:
taipei = Market([2., 1., 2.], [0.2, 0.3, 0.1])
p_star = quickeq(taipei, demand)

3-element Vector{Float64}:
 0.6264241117723929
 0.40000000000976904
 0.7632120558895371

The demand function is *piecewise linear* in $p$ (piecewise because the ordering of the $p_c$ values changes). Expanding the equation above, we can see that the demand vector is defined by the system
$$ D = A p + \gamma$$
where 
$$A \equiv \begin{bmatrix}
\gamma_1 \left( \frac{-1}{\gamma_1} \right) & \gamma_1 \left(\frac{1}{\gamma_1} - \frac{1}{\gamma_1 + \gamma_2} \right) & \gamma_1 \left(\frac{1}{\gamma_1 + \gamma_2} - \frac{1}{\gamma_1 + \gamma_2 + \gamma_3} \right) & \cdots &  \gamma_1 \left(\frac{1}{\sum_{i=1}^{|C| - 1}\gamma_i} - 1 \right)  \\
0 & \gamma_2 \left( \frac{-1}{\gamma_1 + \gamma_2} \right) & \gamma_2 \left(\frac{1}{\gamma_1 + \gamma_2} - \frac{1}{\gamma_1 + \gamma_2 + \gamma_3} \right) & \cdots &  \gamma_2 \left(\frac{1}{\sum_{i=1}^{|C| - 1}\gamma_i} - 1 \right)  \\
0 & 0 & \gamma_3 \left( \frac{-1}{\gamma_1 + \gamma_2 + \gamma_3} \right) & \cdots &  \gamma_3 \left(\frac{1}{\sum_{i=1}^{|C| - 1}\gamma_i} - 1 \right)  \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & 0 &  \gamma_{|C|} \left(\frac{1}{\sum_{i=1}^{|C| - 1}\gamma_i} - 1 \right)  \\
\end{bmatrix}$$
where in the final column I have applied the assumption that $\sum_{i=1}^{|C| }\gamma_i = 1$. Assuming $\gamma > 0$, $A$ is strictly triangular, hence invertible. 

In [15]:
function A_gamma(market, cutoffs)
    m = length(market.qualities)
    A = UpperTriangular(zeros(m, m))
    
    for c in 1:m
        A[m, m] = γ

LoadError: syntax: incomplete: "function" at In[15]:1 requires end

In [13]:
UpperTriangular(zeros(12 ,12))

12×12 UpperTriangular{Float64, Matrix{Float64}}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  ⋅   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  ⋅    ⋅   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  ⋅    ⋅    ⋅   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  ⋅    ⋅    ⋅    ⋅   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  ⋅    ⋅    ⋅    ⋅    ⋅   0.0  0.0  0.0  0.0  0.0  0.0  0.0
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   0.0  0.0  0.0  0.0  0.0  0.0
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   0.0  0.0  0.0  0.0  0.0
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   0.0  0.0  0.0  0.0
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   0.0  0.0  0.0
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   0.0  0.0
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   0.0